<a href="https://www.kaggle.com/code/averma111/spaceship-titanic-wip?scriptVersionId=124718836" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import seaborn as sns
from matplotlib import pyplot as plt
import pandas as pd

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


In [3]:
ROOT_PATH = '/kaggle/input/spaceship-titanic/'
TRAIN_FILE = 'train.csv'
TEST_FILE = 'test.csv'

In [6]:
train_data = pd.read_csv(ROOT_PATH+TRAIN_FILE)
print(f'Number of rows = {train_data.shape[0]} and Number of cols ={train_data.shape[1]}')

Number of rows = 8693 and Number of cols =14


In [7]:
test_data = pd.read_csv(ROOT_PATH+TEST_FILE)
print(f'Number of rows = {test_data.shape[0]} and Number of cols ={test_data.shape[1]}')

Number of rows = 4277 and Number of cols =13
